# RAG 평가를 위한 합성 데이터셋 생성


RAG(검색 증강 생성) 강화 파이프라인의 성능을 평가하는 것은 매우 중요합니다.
하지만 문서에서 수백 개의 QA(질문-답변-컨텍스트) 샘플을 수동으로 만드는 것은 시간이 많이 소요되고 노동 집약적입니다. 또한 사람이 생성한 질문은 철저한 평가에 필요한 복잡성 수준에 도달하는 데 어려움이 있을 수 있어, 결과적으로 평가의 품질에 영향을 미칠 수 있습니다.

### 참조
- [Testset Generation for RAG](https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/)

---




## 0. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)


module_path = ".."
add_python_path(module_path)


from libs.generate_synthesis_data import (
    PDFLoader,split_text,add_meta_into_chunk,
    create_prompt,
    converse_with_bedrock_tools,
    parse_tool_use,
    generate_qa_dataset,
)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock/notebook', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python310.zip', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/lib-dynload', '', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock']


## 1. 실습에 사용된 문서

- 파일 위치: ../data/241231-Sonnet-recipe.pdf
- 이 문서는 Claude Sonnet3.5 로 생성된 데이타를 pdf 파일로 생성 했습니다.


In [3]:
loader = PDFLoader("../data/241231-Sonnet-recipe.pdf", start_page=1, end_page=3)
docs = loader.load()
docs

{'page_content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파두부가 있는데, 두부와 다진 고기를 매운 두반장 소스에 볶아 만\n듭니다. 깐풍기는 닭고기를 튀겨서 매콤달콤한 소스를 絡めて만드는 요리이고, 탕\n수육은 돼지고기를 튀겨서 새콤달콤한 소스를 부어 먹습니다.\n태국 요리로는 팟타이가 유명한데, 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려\n고소한 맛을 냅니다. 그린커리는 코코넛밀크와 향신료를 넣어 만든 커리에 닭고기\n와 가지를 넣어 끓입니다. 톰얌쿵은 새우와 버섯을 레몬그라스, 카피르라임 등의\n향신료와 함께 시큼매콤하게 끓이는 수프입니다.\n베트남 요리에서는 쌀국수인 포가 대표적인데, 소고기 육수에 쌀국수와 고기를 넣\n고 고수, 숙주나물을 곁들여 먹습니다. 반미는 바게트빵에 각종 채소와 고기를\n넣어 만드는 샌드위치이며, 분짜는 구운 돼지고기와 쌀국수를 느억맘 소스에 찍어\n먹는 요리입니다.\n이탈리아 요리로 더 들어가보면, 리

## 2.  문서를 Chunk 로 분리 및 메타 데이터 삽입

In [7]:
chunks = split_text(
                text = docs['page_content'], 
                chunk_size=512, 
                chunk_overlap=0)
print("length of chunks: ", len(chunks))
chunks[0]

length of chunks:  8


'볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n'

In [8]:
chunks_with_metadata = add_meta_into_chunk(chunks, docs)    
chunks_with_metadata[0]

{'content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n',
 'metadata': {'chunk_id': 0, 'filename': '../data/241231-Sonnet-recipe.pdf'}}

## 3. Test Q&A Dataset Generation

### 3.1. Amazon Bedrock 사용위한 설정

In [9]:
import boto3
from botocore.config import Config

region = 'us-west-2'
retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

### 3.2. 파라미터 설정 및 생성

In [10]:
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
num_of_generated_sample = 10
output_file="../data/sample_qa_dataset.jsonl"

In [11]:
%%time

generate_qa_dataset(
                    model_id = model_id, 
                    chunks = chunks_with_metadata, 
                    num_pairs = num_of_generated_sample, 
                    boto3_client = boto3_client,
                    output_file= output_file,
                    # verbose=True,
                    verbose=False,
)

{'question': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?', 'ground_truth': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성은 다음과 같습니다:\n\n1. 프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 채식 요리로, 채식주의자에게 적합합니다.\n\n2. 스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자도 즐길 수 있는 상큼한 요리입니다.\n\n3. 그리스의 무사카: 가지와 다진 고기를 층층이 쌓아 만드는 요리로, 육식주의자를 위한 선택입니다. 채식 버전으로 변형도 가능합니다.\n\n4. 모로코의 쿠스쿠스: 밀가루로 만든 알갱이에 채소를 얹어 먹는 요리로, 고기를 별도로 제공하여 선택적으로 먹을 수 있게 합니다.\n\n5. 스페인의 빠에야: 해산물과 채소를 넣은 쌀 요리로, 해산물을 먹는 사람과 채식주의자 모두에게 적합합니다.\n\n이러한 구성은 다양한 식단 선호도를 고려하면서도 지중해 요리의 특색을 잘 보여줍니다. 채식 옵션과 육류 옵션을 동시에 제공하여 모든 식성의 사람들을 만족시킬 수 있으며, 각 요리의 재료와 조리법을 조절하여 개인의 선호에 맞출 수 있는 유연성도 갖추고 있습니다.', 'question_type': 'complex', 'contexts': '프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통 요리입니다. 와인의 풍미가 닭고기에 깊이 배어들어 독특한 맛을 자아내며,\n양파와 버섯, 베이컨을 함께 넣어 풍성한 맛을 더합니다. 라타투이는 가지, 토마\n토, 호박, 파프리카 등 지중해 채소들을 올리브유로 천천히 익혀 만드는 프로방\n스 지방의 대표적인 채식 요리입니다. 끼슈는 버터가 듬뿍 들어간 파이 반죽 위\n에 달걀과 크림을 섞은 커스터드를 붓고 베이컨, 치즈, 채소를 넣어 구워내는 고

[{'question': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성을 제안한다면, 어떤 요리들을 선택하고 그 이유는 무엇인가요?',
  'ground_truth': '지중해 연안 국가들의 요리 중 채식주의자와 육식주의자 모두를 만족시킬 수 있는 메뉴 구성은 다음과 같습니다:\n\n1. 프랑스의 라타투이: 가지, 토마토, 호박, 파프리카 등 다양한 채소로 만든 채식 요리로, 채식주의자에게 적합합니다.\n\n2. 스페인의 가스파초: 토마토, 오이, 피망으로 만든 차가운 스프로, 채식주의자도 즐길 수 있는 상큼한 요리입니다.\n\n3. 그리스의 무사카: 가지와 다진 고기를 층층이 쌓아 만드는 요리로, 육식주의자를 위한 선택입니다. 채식 버전으로 변형도 가능합니다.\n\n4. 모로코의 쿠스쿠스: 밀가루로 만든 알갱이에 채소를 얹어 먹는 요리로, 고기를 별도로 제공하여 선택적으로 먹을 수 있게 합니다.\n\n5. 스페인의 빠에야: 해산물과 채소를 넣은 쌀 요리로, 해산물을 먹는 사람과 채식주의자 모두에게 적합합니다.\n\n이러한 구성은 다양한 식단 선호도를 고려하면서도 지중해 요리의 특색을 잘 보여줍니다. 채식 옵션과 육류 옵션을 동시에 제공하여 모든 식성의 사람들을 만족시킬 수 있으며, 각 요리의 재료와 조리법을 조절하여 개인의 선호에 맞출 수 있는 유연성도 갖추고 있습니다.',
  'question_type': 'complex',
  'contexts': '프랑스 요리로 더 살펴보면, 코코뱅은 닭고기를 레드와인에 장시간 끓여 만드는\n전통 요리입니다. 와인의 풍미가 닭고기에 깊이 배어들어 독특한 맛을 자아내며,\n양파와 버섯, 베이컨을 함께 넣어 풍성한 맛을 더합니다. 라타투이는 가지, 토마\n토, 호박, 파프리카 등 지중해 채소들을 올리브유로 천천히 익혀 만드는 프로방\n스 지방의 대표적인 채식 요리입니다. 끼슈는 버터가 듬뿍 들어간 파이 반죽 위\n에 달걀과 크림을 섞은 커스터드를 붓고 베이컨, 치즈, 채소를 넣어